In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import preprocessing
import xgboost as xgb
import scipy.stats as stats
import gc

In [13]:
train = pd.read_csv('grupo-bimbo-inventory-demand/train.csv')
test = pd.read_csv('grupo-bimbo-inventory-demand/test.csv')

print('Train and Test Read')

train['target'] = train['Demanda_uni_equil']
train.drop(['Demanda_uni_equil'],axis=1, inplace = True)

train['tst'] = 0
test['tst'] = 1

data = pd.concat([train,test], axis=0, copy=True)

print('Train and Test Concat')
del train
del test
gc.collect()

for i in range(1,6):
    lag = 'Lag' + str(i)
    print('Lag:',lag)

    data1 = data[['Semana','Cliente_ID','Producto_ID','target']]
    data1.loc[:,'Semana'] = data1['Semana'] +i
    data1 = pd.groupby(data1,['Semana','Cliente_ID','Producto_ID']).mean()
    data1 = data1.reset_index()
    data1.rename(columns={'target': lag}, inplace=True)
    data = pd.merge(data,data1,
                    how='left',
                    left_on=['Semana','Cliente_ID','Producto_ID'],
                    right_on=['Semana','Cliente_ID','Producto_ID'],
                    left_index=False, right_index=False, sort=True,
                    suffixes=('_x', '_y'), copy=False, )
    del data1
    gc.collect()

Train and Test Read
Train and Test Concat
Lag: Lag1


/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_26852/3371105216.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.loc[:,'Semana'] = data1['Semana'] +i


AttributeError: module 'pandas' has no attribute 'groupby'

In [12]:

data['TotalLags'] = data['Lag1'] + data['Lag2']+ data['Lag3']+ data['Lag4']+ data['Lag5']

data=data[data['Semana']>8]  # NOW I WORK WITH WEEKS 9,10,11

nAgencia = pd.DataFrame(pd.groupby(data,['Agencia_ID','Semana'])['target'].count())
nAgencia = nAgencia.reset_index()
nAgencia.rename(columns={'target': 'nAgencia'}, inplace=True)
nAgencia = pd.DataFrame(pd.groupby(nAgencia,['Agencia_ID'])['nAgencia'].mean())
nAgencia = nAgencia.reset_index()


data = pd.merge(data, nAgencia,
                how='left',
                left_on=['Agencia_ID'],
                right_on=['Agencia_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nAgencia
gc.collect()
print('merge completo nAgencia')
print(data.shape[0])

nRuta_SAK = pd.DataFrame(pd.groupby(data,['Ruta_SAK','Semana'])['target'].count())
nRuta_SAK = nRuta_SAK.reset_index()
nRuta_SAK.rename(columns={'target': 'nRuta_SAK'}, inplace=True)
nRuta_SAK = pd.DataFrame(pd.groupby(nRuta_SAK,['Ruta_SAK'])['nRuta_SAK'].mean())
nRuta_SAK = nRuta_SAK.reset_index()


data = pd.merge(data, nRuta_SAK,
                how='left',
                left_on=['Ruta_SAK'],
                right_on=['Ruta_SAK'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nRuta_SAK
gc.collect()
print('merge completo nRuta_SAK')
print(data.shape[0])

nCliente_ID = pd.DataFrame(pd.groupby(data,['Cliente_ID','Semana'])['target'].count())
nCliente_ID = nCliente_ID.reset_index()
nCliente_ID.rename(columns={'target': 'nCliente_ID'}, inplace=True)
nCliente_ID = pd.DataFrame(pd.groupby(nCliente_ID,['Cliente_ID'])['nCliente_ID'].mean())
nCliente_ID = nCliente_ID.reset_index()


data = pd.merge(data, nCliente_ID,
                how='left',
                left_on=['Cliente_ID'],
                right_on=['Cliente_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nCliente_ID
gc.collect()
print('merge completo nCliente_ID')
print(data.shape[0])

nProducto_ID = pd.DataFrame(pd.groupby(data,['Producto_ID','Semana'])['target'].count())
nProducto_ID = nProducto_ID.reset_index()
nProducto_ID.rename(columns={'target': 'nProducto_ID'}, inplace=True)
nProducto_ID = pd.DataFrame(pd.groupby(nProducto_ID,['Producto_ID'])['nProducto_ID'].mean())
nProducto_ID = nProducto_ID.reset_index()


data = pd.merge(data, nProducto_ID,
                how='left',
                left_on=['Producto_ID'],
                right_on=['Producto_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nProducto_ID
gc.collect()
print('merge completo nProducto_ID')
print(data.shape[0])

data.replace(np.nan,0, inplace=True)

train = data[data['tst']==0]
predict = data[data['tst']==1]

train['target'] = np.log(train['target'] + 1)
#train2 = train.sample(n=1000000)   <-- another possible reduction of data for fast testing
train2 = train
y = train['target']
X = train[[  'Agencia_ID','Canal_ID','Cliente_ID','Producto_ID','Ruta_SAK',
             'Lag1','Lag2','Lag3','Lag4','Lag5','TotalLags',
             'nAgencia','nRuta_SAK','nCliente_ID','nProducto_ID']]

print(X.shape, y.shape)

Train and Test Read
Train and Test Concat
Lag: Lag1


/var/folders/zs/yzy0xdy166x3hv5mvqzqrnlm0000gn/T/ipykernel_26852/2108532917.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.loc[:,'Semana'] = data1['Semana'] +i


AttributeError: module 'pandas' has no attribute 'groupby'

In [9]:

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=1729)
print(X_train.shape, X_test.shape)